In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['stopwords_eng.txt', 'valid_lyrics_rem_200.csv', 'train_lyrics_rem_1000.csv', 'valid_lyrics_200.csv', 'train_lyrics_1000.csv', 'train_1000.csv', 'label_encoder.p.txt', 'stopwords.p']


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import seaborn as sns
train = pd.read_csv('../input/train_lyrics_1000.csv', usecols=range(7))
validate =pd.read_csv('../input/valid_lyrics_200.csv')
print(df1.head())
df.head()

NameError: name 'df1' is not defined

**1 FEATURE EXTRACTION**

**1.1 Number of Words**
One of the most basic features we can extract is the number of words in each lyrics. The basic intuition behind this is that generally, the negative sentiments contain a lesser amount of words than the positive ones.[](http://)

In [4]:
train['word_count'] = train['lyrics'].apply(lambda x: len(str(x).split(" ")))
train[['lyrics','word_count']].head()

,lyrics,word_count
0,Verse One:\n\nAlright I might\nHave had a litt...,392
1,Adam Ant/Marco Pirroni\nEvery girl is a someth...,114
2,"I've just erased it's been a while, I've got a...",110
3,Little darling \nWhere you've been so long \nI...,183
4,"Lead Vocal by Greg\n\nWell, these late night c...",182


**1.2 Number of characters**
This feature is also based on the previous feature intuition. Here, we calculate the number of characters in each lyrics. This is done by calculating the length of the tweet.

In [5]:
train['char_count'] = train['lyrics'].str.len() ## this also includes spaces
train[['lyrics','char_count']].head()

,lyrics,char_count
0,Verse One:\n\nAlright I might\nHave had a litt...,2181
1,Adam Ant/Marco Pirroni\nEvery girl is a someth...,713
2,"I've just erased it's been a while, I've got a...",600
3,Little darling \nWhere you've been so long \nI...,977
4,"Lead Vocal by Greg\n\nWell, these late night c...",1130


**1.3 Average Word Length**
We will also extract another feature which will calculate the average word length of each lyrics. This can also potentially help us in improving our model.

Here, we simply take the sum of the length of all the words and divide it by the total length of the tweet:

In [6]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

train['avg_word'] = train['lyrics'].apply(lambda x: avg_word(x))
train[['lyrics','avg_word']].head()

,lyrics,avg_word
0,Verse One:\n\nAlright I might\nHave had a litt...,3.742358
1,Adam Ant/Marco Pirroni\nEvery girl is a someth...,4.211679
2,"I've just erased it's been a while, I've got a...",3.869919
3,Little darling \nWhere you've been so long \nI...,4.147541
4,"Lead Vocal by Greg\n\nWell, these late night c...",4.136986


**1.4 Number of stopwords**
Generally, while solving an NLP problem, the first thing we do is to remove the stopwords. But sometimes calculating the number of stopwords can also give us some extra information which we might have been losing before.

Here, we have imported stopwords from NLTK, which is a basic NLP library in python.

In [7]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

train['stopwords'] = train['lyrics'].apply(lambda x: len([x for x in x.split() if x in stop]))
train[['lyrics','stopwords']].head()

,lyrics,stopwords
0,Verse One:\n\nAlright I might\nHave had a litt...,163
1,Adam Ant/Marco Pirroni\nEvery girl is a someth...,48
2,"I've just erased it's been a while, I've got a...",47
3,Little darling \nWhere you've been so long \nI...,66
4,"Lead Vocal by Greg\n\nWell, these late night c...",78


**1.5 Number of numerics**
Just like we calculated the number of words, we can also calculate the number of numerics which are present in the lyrics. It does not have a lot of use in our example, but this is still a useful feature that should be run while doing similar exercises. For example, 

In [8]:
train['numerics'] = train['lyrics'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
train[['lyrics','numerics']].head()

,lyrics,numerics
0,Verse One:\n\nAlright I might\nHave had a litt...,0
1,Adam Ant/Marco Pirroni\nEvery girl is a someth...,0
2,"I've just erased it's been a while, I've got a...",0
3,Little darling \nWhere you've been so long \nI...,0
4,"Lead Vocal by Greg\n\nWell, these late night c...",1


**2. Basic Pre-processing**
So far, we have learned how to extract basic features from text data. Before diving into text and feature extraction, our first step should be cleaning the data in order to obtain better features. We will achieve this by doing some of the basic pre-processing steps on our training data.

So, let’s get into it.

**2.1 Lower case**
The first pre-processing step which we will do is transform our lyrics into lower case. This avoids having multiple copies of the same words. For example, while calculating the word count, ‘Analytics’ and ‘analytics’ will be taken as different words.

In [9]:
train['lyrics'] = train['lyrics'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train['lyrics'].head()

0    verse one: alright i might have had a little g...
1    adam ant/marco pirroni every girl is a somethi...
2    i've just erased it's been a while, i've got a...
3    little darling where you've been so long i've ...
4    lead vocal by greg well, these late night conv...
Name: lyrics, dtype: object

**2.2 Removing Punctuation**
The next step is to remove punctuation, as it doesn’t add any extra information while treating text data. Therefore removing all instances of it will help us reduce the size of the training data.

In [10]:
train['lyrics'] = train['lyrics'].str.replace('[^\w\s]','')
train['lyrics'].head()

0    verse one alright i might have had a little gl...
1    adam antmarco pirroni every girl is a somethin...
2    ive just erased its been a while ive got a wor...
3    little darling where youve been so long ive be...
4    lead vocal by greg well these late night conve...
Name: lyrics, dtype: object

**2.3 Removal of Stop Words**
As we discussed earlier, stop words (or commonly occurring words) should be removed from the text data. For this purpose, we can either create a list of stopwords ourselves or we can use predefined libraries.

In [11]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
train['lyrics'] = train['lyrics'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train['lyrics'].head()

0    verse one alright might little glare stared ya...
1    adam antmarco pirroni every girl something gir...
2    ive erased ive got world sale walk away better...
3    little darling youve long ive thinking ya feel...
4    lead vocal greg well late night conversations ...
Name: lyrics, dtype: object

**2.4 Rare words removal**
 we can remove the most common words as well as rare words, this time let’s remove rarely occurring words from the text. Because they’re so rare, the association between them and other words is dominated by noise. You can replace rare words with a more general form and then this will have higher counts

In [12]:
freq = pd.Series(' '.join(train['lyrics']).split()).value_counts()[-10:]
freq

brace        1
trades       1
noo          1
worryin      1
buckwild     1
portafide    1
plugged      1
pretended    1
withyeah     1
angelic      1
dtype: int64

**2.5 Tokenization**
Tokenization refers to dividing the text into a sequence of words or sentences. In our example, we have used the textblob library to first transform our lyrics into a blob and then converted them into a series of words.

In [13]:
from textblob import TextBlob
TextBlob(train['lyrics'][1]).words

WordList(['adam', 'antmarco', 'pirroni', 'every', 'girl', 'something', 'girl', 'every', 'girl', 'something', 'girl', 'every', 'girl', 'something', 'girl', 'shes', 'got', 'something', 'every', 'girl', 'something', 'girl', 'every', 'girl', 'somehing', 'smile', 'every', 'girl', 'something', 'girl', 'three', 'wishes', 'know', 'mine', 'would', 'ask', 'stars', 'shine', 'wouldnt', 'ask', 'gems', 'pearls', 'id', 'ask', 'something', 'girls', 'every', 'girl', 'something', 'girl', 'whites', 'eyes', 'tip', 'curl', 'every', 'girl', 'sends', 'something', 'boy', 'something', 'old', 'world', 'wont', 'treat', 'right', 'youre', 'feeling', 'tonight', 'think', 'someone', 'relax', 'let', 'something', 'every', 'girl', 'something', 'girl', 'shes', 'got', 'shes', 'got', 'something'])

**2.6 Stemming**
Stemming refers to the removal of suffices, like “ing”, “ly”, “s”, etc. by a simple rule-based approach. For this purpose, we will use PorterStemmer from the NLTK library.

In [14]:
from nltk.stem import PorterStemmer
st = PorterStemmer()
train['lyrics'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0    vers one alright might littl glare stare ya ho...
1    adam antmarco pirroni everi girl someth girl e...
2    ive eras ive got world sale walk away better d...
3    littl darl youv long ive think ya feel real st...
4    lead vocal greg well late night convers leav s...
Name: lyrics, dtype: object

**2.7 Lemmatization**
Lemmatization is a more effective option than stemming because it converts the word into its root word, rather than just stripping the suffices. It makes use of the vocabulary and does a morphological analysis to obtain the root word. Therefore, we usually prefer using lemmatization over stemming.

In [15]:
from textblob import Word
train['lyrics'] = train['lyrics'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
train['lyrics'].head()

0    verse one alright might little glare stared ya...
1    adam antmarco pirroni every girl something gir...
2    ive erased ive got world sale walk away better...
3    little darling youve long ive thinking ya feel...
4    lead vocal greg well late night conversation l...
Name: lyrics, dtype: object

**3. Advance Text Processing**
Up to this point, we have done all the basic pre-processing steps in order to clean our data. Now, we can finally move on to extracting features using NLP techniques.

**3.1 N-grams**
N-grams are the combination of multiple words used together. Ngrams with N=1 are called unigrams. Similarly, bigrams (N=2), trigrams (N=3) and so on can also be used.

Unigrams do not usually contain as much information as compared to bigrams and trigrams. The basic principle behind n-grams is that they capture the language structure, like what letter or word is likely to follow the given one. The longer the n-gram (the higher the n), the more context you have to work with. Optimum length really depends on the application – if your n-grams are too short, you may fail to capture important differences. On the other hand, if they are too long, you may fail to capture the “general knowledge” and only stick to particular cases.

So, let’s quickly extract bigrams from our tweets using the ngrams function of the textblob library.

In [16]:
TextBlob(train['lyrics'][0]).ngrams(2)

[WordList(['verse', 'one']),
 WordList(['one', 'alright']),
 WordList(['alright', 'might']),
 WordList(['might', 'little']),
 WordList(['little', 'glare']),
 WordList(['glare', 'stared']),
 WordList(['stared', 'ya']),
 WordList(['ya', 'ho']),
 WordList(['ho', 'didnt']),
 WordList(['didnt', 'know']),
 WordList(['know', 'like']),
 WordList(['like', 'stared']),
 WordList(['stared', 'right']),
 WordList(['right', 'back']),
 WordList(['back', 'nigga']),
 WordList(['nigga', 'warnin']),
 WordList(['warnin', 'comin']),
 WordList(['comin', 'react']),
 WordList(['react', 'like']),
 WordList(['like', 'mack']),
 WordList(['mack', 'act']),
 WordList(['act', 'cool']),
 WordList(['cool', 'test']),
 WordList(['test', 'cause']),
 WordList(['cause', 'aint']),
 WordList(['aint', 'jester']),
 WordList(['jester', 'suggest']),
 WordList(['suggest', 'friend']),
 WordList(['friend', 'outtie']),
 WordList(['outtie', 'cause']),
 WordList(['cause', 'dont']),
 WordList(['dont', 'want']),
 WordList(['want', 'make'

**3.2 Term frequency**
Term frequency is simply the ratio of the count of a word present in a sentence, to the length of the sentence.

Therefore, we can generalize term frequency as:

TF = (Number of times term T appears in the particular row) / (number of terms in that row)

To understand more about Term Frequency

In [17]:
tf1 = (train['lyrics'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

,words,tf
0,girl,17
1,something,13
2,every,9
3,shes,3
4,got,3
5,ask,3
6,sends,1
7,think,1
8,boy,1
9,antmarco,1


**3.3 Inverse Document Frequency**
The intuition behind inverse document frequency (IDF) is that a word is not of much use to us if it’s appearing in all the documents.

Therefore, the IDF of each word is the log of the ratio of the total number of rows to the number of rows in which that word is present.

IDF = log(N/n), where, N is the total number of rows and n is the number of rows in which the word was present.

So, let’s calculate IDF for the same tweets for which we calculated the term frequency.

In [18]:
for i,word in enumerate(tf1['words']):
  tf1.loc[i, 'idf'] = np.log(train.shape[0]/(len(train[train['lyrics'].str.contains(word)])))

tf1

,words,tf,idf
0,girl,17,1.851509
1,something,13,2.396896
2,every,9,1.081755
3,shes,3,2.733368
4,got,3,0.944176
5,ask,3,2.375156
6,sends,1,6.214608
7,think,1,1.523260
8,boy,1,2.137071
9,antmarco,1,6.907755


The more the value of IDF, the more unique is the word.

 

**3.4 Term Frequency – Inverse Document Frequency (TF-IDF)**
TF-IDF is the multiplication of the TF and IDF which we calculated above.

In [19]:
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

,words,tf,idf,tfidf
0,girl,17,1.851509,31.475661
1,something,13,2.396896,31.159645
2,every,9,1.081755,9.735797
3,shes,3,2.733368,8.200104
4,got,3,0.944176,2.832528
5,ask,3,2.375156,7.125467
6,sends,1,6.214608,6.214608
7,think,1,1.523260,1.523260
8,boy,1,2.137071,2.137071
9,antmarco,1,6.907755,6.907755
